# Basic usage of the ``distoptica`` library #

## A NOTE BEFORE STARTING ##

Since the ``distoptica`` git repository tracks this notebook under its original
basename ``basic_usage.ipynb``, we recommend that you copy the original notebook
and rename it to any other basename that is not one of the original basenames
that appear in the ``<root>/examples`` directory before executing any of the
notebook cells below, where ``<root>`` is the root of the ``distoptica``
repository. This way you can explore the notebook by executing and modifying
cells without changing the original notebook, which is being tracked by git.

## Import necessary modules ##

In [ ]:
# For general array handling.
import numpy as np
import torch

# For creating hyperspy signals.
import hyperspy.signals
import hyperspy.axes

# For creating quiver plots.
import matplotlib.pyplot as plt



# The library that is the subject of this demonstration.
import distoptica

In [ ]:
%matplotlib ipympl
%matplotlib ipympl

## Introduction ##

In this notebook, we demonstrate how one can use each function and class in the
``distoptica`` library.

You can find the documentation for the ``distoptica`` library
[here](https://mrfitzpa.github.io/distoptica/_autosummary/distoptica.html).  It 
is recommended that you consult the documentation of this library as you explore
the notebook. Moreover, users should execute the cells in the order that they
appear, i.e. from top to bottom, as some cells reference variables that are set
in other cells above them.

## Using the ``distoptica`` library ##

### Creating an undistorted image set ###

Let's create a 2D ``hyperspy`` signal that stores a set of undistorted images
that we will distort then resample below, and then subsequently undistort then
resample to recover approximately the original image set:

In [ ]:
def generate_undistorted_image_set_signal():
    kwargs = {"data": generate_undistorted_image_set_signal_data(), 
              "metadata": generate_undistorted_image_set_signal_metadata()}
    undistorted_image_set_signal = hyperspy.signals.Signal2D(**kwargs)

    axes = generate_undistorted_image_set_signal_axes()

    for axis_idx, axis in enumerate(axes):
        undistorted_image_set_signal.axes_manager[axis_idx].update_from(axis)
        undistorted_image_set_signal.axes_manager[axis_idx].name = axis.name

    return undistorted_image_set_signal



def generate_undistorted_image_set_signal_data():
    signal_data_shape = generate_undistorted_image_set_signal_data_shape()
    Y_dim, X_dim, v_dim, h_dim = signal_data_shape

    undistorted_image_supports = generate_undistorted_image_supports(h_dim, 
                                                                     v_dim)

    metadata = \
        generate_undistorted_image_set_signal_metadata()
    max_pixel_vals_of_channels_of_undistorted_image = \
        metadata["max_pixel_vals_of_channels_of_undistorted_image"]

    kwargs = {"shape": signal_data_shape, "dtype": "float"}
    signal_data = np.zeros(**kwargs)

    for X_idx in range(X_dim):
        max_pixel_val_of_channel = \
            max_pixel_vals_of_channels_of_undistorted_image[X_idx]
        signal_data[:, X_idx] = \
            (undistorted_image_supports[:, X_idx] * max_pixel_val_of_channel)

    undistorted_image_set_signal_data = signal_data

    return undistorted_image_set_signal_data



def generate_undistorted_image_set_signal_data_shape():
    undistorted_image_set_signal_data_shape = (3, 2, 140, 140)
    
    return undistorted_image_set_signal_data_shape



def generate_undistorted_image_supports(h_dim, v_dim):
    signal_data_shape = generate_undistorted_image_set_signal_data_shape()
    Y_dim, X_dim, _, _ = signal_data_shape

    metadata = \
        generate_undistorted_image_set_signal_metadata()
    undistorted_disk_centers = \
        metadata["undistorted_disk_centers"]
    undistorted_disk_radius = \
        metadata["undistorted_disk_radius"]

    kwargs = {"shape": (Y_dim, X_dim, v_dim, h_dim), "dtype": "bool"}
    undistorted_image_supports = np.zeros(**kwargs)

    u_x, u_y = generate_coord_meshgrid(h_dim, v_dim)

    for Y_idx in range(Y_dim):
        for X_idx in range(X_dim):
            u_x_c, u_y_c = undistorted_disk_centers[Y_idx]
            u_xy = np.sqrt((u_x-u_x_c)**2 + (u_y-u_y_c)**2)
            u_R = undistorted_disk_radius
            undistorted_image_supports[Y_idx, X_idx] = (u_xy <= u_R)

    return undistorted_image_supports



def generate_coord_meshgrid(h_dim, v_dim):
    m_range = np.arange(h_dim)
    n_range = np.arange(v_dim)

    horizontal_coords_of_meshgrid = \
        (m_range + 0.5) / m_range.size
    vertical_coords_of_meshgrid = \
        1 - (n_range + 0.5) / n_range.size

    pair_of_1d_coord_arrays = (horizontal_coords_of_meshgrid,
                               vertical_coords_of_meshgrid)
    coord_meshgrid = np.meshgrid(*pair_of_1d_coord_arrays,
                                 indexing="xy")

    return coord_meshgrid



def generate_undistorted_image_set_signal_metadata():
    metadata = {"General": \
                {"title": "Undistorted Image Set"}, 
                "Signal": \
                dict(), 
                "undistorted_disk_centers": \
                generate_undistorted_disk_centers(),
                "undistorted_disk_radius": \
                generate_undistorted_disk_radius(), 
                "max_pixel_vals_of_channels_of_undistorted_image": \
                generate_max_pixel_vals_of_channels_of_undistorted_image()}

    undistorted_image_set_signal_metadata = metadata

    return undistorted_image_set_signal_metadata



def generate_undistorted_disk_centers():
    undistorted_disk_centers = ((0.5, 0.7),
                                (0.5, 0.5),
                                (0.5, 0.3))

    return undistorted_disk_centers



def generate_undistorted_disk_radius():
    undistorted_disk_radius = 1/6

    return undistorted_disk_radius



def generate_max_pixel_vals_of_channels_of_undistorted_image():
    max_pixel_vals_of_channels_of_undistorted_image = (1, 3)

    return max_pixel_vals_of_channels_of_undistorted_image



def generate_undistorted_image_set_signal_axes():
    signal_data_shape = generate_undistorted_image_set_signal_data_shape()
    Y_dim, X_dim, v_dim, h_dim = signal_data_shape

    d_h = 1/h_dim
    d_v = -1/v_dim

    axes_sizes = (X_dim, Y_dim, h_dim, v_dim)
    axes_scales = (1, 1, d_h, d_v)
    axes_offsets = (0, 0, 0.5*d_h, 1+0.5*d_v)
    axes_names = ("$X$", 
                  "$Y$", 
                  "fractional horizontal coordinate", 
                  "fractional vertical coordinate")

    axes = tuple()
    for axis_idx, _ in enumerate(axes_names):
        kwargs = {"size": axes_sizes[axis_idx],
                  "scale": axes_scales[axis_idx],
                  "offset": axes_offsets[axis_idx],
                  "name": axes_names[axis_idx]}
        axis = hyperspy.axes.UniformDataAxis(**kwargs)
        axes += (axis,)

    undistorted_image_set_signal_axes = axes

    return undistorted_image_set_signal_axes



undistorted_image_set_signal = generate_undistorted_image_set_signal()

Let's visualize the undistorted image set that we just created.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
undistorted_image_set_signal.plot(**kwargs)

### Specifying the distortion model ###

In order to distort and/or undistort images, we need to specify a distortion
model. This entails specifying several sets of parameters.

First, we need to specify the coordinate transformation that maps fractional
coordinates of an undistorted image to those of the corresponding distorted
image of interest. See
[here](https://mrfitzpa.github.io/distoptica/_autosummary/distoptica.CoordTransformParams.html)
for a detailed description of the mathematical form of the coordinate
transformation, and the parameters required to specify an instance of such a
coordinate transformation. In short, we need to specify the distortion center,
and four coefficient matrices: the radial cosine, the radial sine, the
tangential cosine, and the tangential sine coefficient matrices. In this
demonstration, we will chose coefficient matrices corresponding to a distortion
field that is a combination of elliptical, quadratic radial, parabolic, and
spiral distortion:

In [ ]:
center = (0.52, 0.49)

quadratic_radial_distortion_amplitude = -0.4

spiral_distortion_amplitude = 0.1

amplitude = 0.07
phase = 7*np.pi/8
elliptical_distortion_vector = (amplitude*np.cos(2*phase).item(), 
                                amplitude*np.sin(2*phase).item())

amplitude = 0.1
phase = 4*np.pi/3
parabolic_distortion_vector = (amplitude*np.cos(phase), 
                               amplitude*np.sin(phase))



A_r_0_2 = quadratic_radial_distortion_amplitude
A_r_1_1 = parabolic_distortion_vector[0]
A_r_2_0 = elliptical_distortion_vector[0]

radial_cosine_coefficient_matrix = ((0.00000, 0.00000, A_r_0_2),
                                    (0.00000, A_r_1_1, 0.00000),
                                    (A_r_2_0, 0.00000, 0.00000))



B_r_0_1 = parabolic_distortion_vector[1]
B_r_1_0 = elliptical_distortion_vector[1]

radial_sine_coefficient_matrix = ((0.00000, B_r_0_1),
                                  (B_r_1_0, 0.00000))


    
A_t_0_2 = spiral_distortion_amplitude
A_t_1_1 = B_r_0_1 / 3
A_t_2_0 = B_r_1_0

tangential_cosine_coefficient_matrix = ((0.00000, 0.00000, A_t_0_2),
                                        (0.00000, A_t_1_1, 0.00000),
                                        (A_t_2_0, 0.00000, 0.00000))


    
B_t_0_1 = -A_r_1_1 / 3
B_t_1_0 = -A_r_2_0

tangential_sine_coefficient_matrix = ((0.00000, B_t_0_1),
                                      (B_t_1_0, 0.00000))



kwargs = {"center": \
          center,
          "radial_cosine_coefficient_matrix": \
          radial_cosine_coefficient_matrix,
          "radial_sine_coefficient_matrix": \
          radial_sine_coefficient_matrix, 
          "tangential_cosine_coefficient_matrix": \
          tangential_cosine_coefficient_matrix,
          "tangential_sine_coefficient_matrix": \
          tangential_sine_coefficient_matrix}
coord_transform_params = distoptica.CoordTransformParams(**kwargs)

Note that the class ``distoptica.CoordTransformParams`` is a subclass of
``fancytypes.PreSerializableAndUpdatable``, meaning that it is a type that is
pre-serializable, that can be constructed from a serializable representation,
and that has an updatable subset of attributes. See
[here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.PreSerializableAndUpdatable.html)
for a definition of pre-serialization, and the documentation for all the
attributes and methods associated with the class
``fancytypes.PreSerializableAndUpdatable``.

Next, we must specify the parameters of the least-squares algorithm to be used
to calculate the right-inverse of the coordinate transformation. See
[here](https://mrfitzpa.github.io/distoptica/_autosummary/distoptica.LeastSquaresAlgParams.html)
for a discussion on the algorithm and a description of the algorithm's
parameters. The values we use for the parameters below should work for most
scenarios.

In [ ]:
kwargs = {"max_num_iterations": 20,
          "initial_damping": 1e-3,
          "factor_for_decreasing_damping": 9,
          "factor_for_increasing_damping": 11,
          "improvement_tol": 0.1, 
          "rel_err_tol": 1e-2, 
          "plateau_tol": 1e-3, 
          "plateau_patience": 2, 
          "skip_validation_and_conversion": False}
least_squares_alg_params = distoptica.LeastSquaresAlgParams(**kwargs)

Note that the class ``distoptica.LeastSquaresAlgParams`` is also a subclass of
``fancytypes.PreSerializableAndUpdatable``.

Next, we must specify the sampling grid dimensions, in units of pixels. Upon
distorting or undistorting an image set, the transformed image set is resampled
at a resolution determined by the sampling grid dimensions. See the summary
documentation of the class ``distoptica.DistortionModel``
[here](https://mrfitzpa.github.io/distoptica/_autosummary/distoptica.DistortionModel.html),
wherein there is a full discussion on how image transformation, and subsequent
resampling is performed.

In [ ]:
sampling_grid_dims_in_pixels = (200,  # Number of columns.
                                200)  # Number of rows.

And lastly, we specify the device to be used to perform computationally
intensive calls to PyTorch functions and where to store attributes of the type
``torch.Tensor``:

In [ ]:
device_name = "cpu"

Putting all the parameters together, we construct our distortion model:

In [ ]:
kwargs = {"coord_transform_params": coord_transform_params,
          "sampling_grid_dims_in_pixels": sampling_grid_dims_in_pixels,
          "device_name": device_name,
          "least_squares_alg_params": least_squares_alg_params}
distortion_model = distoptica.DistortionModel(**kwargs)

Note that the class ``distoptica.DistortionModel`` is also a subclass of
``fancytypes.PreSerializableAndUpdatable``.

One can check whether the distortion model is azimuthally symmetric:

In [ ]:
distortion_model.is_azimuthally_symmetric

An example of a distortion model that is azimuthally symmetric is one that
possesses only quadratic radial distortion.

One can also check whether the distortion model is trivial, i.e. that the
corresponding coordinate transformation is equivalent to the identity
transformation:

In [ ]:
distortion_model.is_trivial

One can also check whether the distortion model is "standard". We define a
standard distortion model as one that possesses any combination of elliptical,
quadratic radial, parabolic, and spiral distortion.

In [ ]:
distortion_model.is_standard

Evidently, the distortion model that we have constructed is standard. See
[here](https://mrfitzpa.github.io/distoptica/_autosummary/distoptica.StandardCoordTransformParams.html)
for a discussion on the mathematical form of the coordinate transformations that
correspond to standard distortion models.

Note that we can get the same information from the object
``coord_transform_params``, which we used to construct the distortion model:

In [ ]:
coord_transform_params.is_corresponding_model_azimuthally_symmetric

In [ ]:
coord_transform_params.is_corresponding_model_trivial

In [ ]:
coord_transform_params.is_corresponding_model_standard

``distoptica`` offers a more convenient way to construct standard distortion
models. First, we need to specify the "standard" coordinate transformation
parameters:

In [ ]:
kwargs = \
    {"center": \
     center,
     "quadratic_radial_distortion_amplitude": \
     quadratic_radial_distortion_amplitude,
     "elliptical_distortion_vector": \
     elliptical_distortion_vector,
     "spiral_distortion_amplitude": \
     spiral_distortion_amplitude,
     "parabolic_distortion_vector": \
     parabolic_distortion_vector}
standard_coord_transform_params = \
    distoptica.StandardCoordTransformParams(**kwargs)

Note that the class ``distoptica.StandardCoordTransformParams`` is also a
subclass of ``fancytypes.PreSerializableAndUpdatable``. Moreover, using the
instance of this class that we have just constructed, we can also determine
whether the corresponding distortion model is azimuthally symmetric, and whether
it is trivial:

In [ ]:
standard_coord_transform_params.is_corresponding_model_azimuthally_symmetric

In [ ]:
standard_coord_transform_params.is_corresponding_model_trivial

After constructing the instance of the class
``distoptica.StandardCoordTransformParams``, we can construct the same
distortion model as before in a similar manner:

In [ ]:
kwargs = {"coord_transform_params": standard_coord_transform_params,
          "sampling_grid_dims_in_pixels": sampling_grid_dims_in_pixels,
          "device_name": device_name,
          "least_squares_alg_params": least_squares_alg_params}
distortion_model = distoptica.DistortionModel(**kwargs)

Alternatively, we can use the function
``distoptica.generate_standard_distortion_model`` to construct the same
distortion model as before:

In [ ]:
kwargs = {"standard_coord_transform_params": standard_coord_transform_params,
          "sampling_grid_dims_in_pixels": sampling_grid_dims_in_pixels,
          "device_name": device_name,
          "least_squares_alg_params": least_squares_alg_params}
distortion_model = distoptica.generate_standard_distortion_model(**kwargs)

As a side note, users can also convert
``distoptica.StandardCoordTransformParams`` objects to
``distoptica.CoordTransformParams`` objects as follows:

In [ ]:
kwargs = \
    {"standard_coord_transform_params": standard_coord_transform_params}
coord_transform_params = \
    distoptica.from_standard_to_generic_coord_transform_params(**kwargs)

where ``standard_coord_transform_params`` and ``coord_transform_params`` specify
the same coordinate transformation.

### Visualizing the convergence of the least-squares algorithm ###

The distortion model that we have just constructed, ``distortion_model``,
samples the flow fields of both the corresponding coordinate transformation, and
its right-inverse, and stores the sampled flow fields as attributes. Recall from
the discussion above that a least-squares algorithm is employed to calculate the
sampled right-inverse of the coordinate transformation, and thus the
corresponding flow field as well. In general, the right-inverse of the
coordinate transformation will not be necessarily well-defined for all points on
the sampling grid, which means that the least-squares algorithm will not
converge for such points. A map of the convergence is stored in the attribute
``convergence_map_of_distorted_then_resampled_images``. Let's visualize this
convergence map.

In [ ]:
def convert_torch_tensor_to_signal(torch_tensor, title):
    numpy_array = torch_tensor.detach().numpy()

    metadata = {"General": {"title": title}, "Signal": dict()}
    
    kwargs = {"data": numpy_array, "metadata": metadata}
    signal = hyperspy.signals.Signal2D(**kwargs)

    num_axes = len(numpy_array.shape)
    
    v_dim, h_dim = numpy_array.shape[-2:]

    d_h = 1/h_dim
    d_v = -1/v_dim

    axes_sizes = (h_dim, v_dim)
    axes_scales = (d_h, d_v)
    axes_offsets = (0.5*d_h, 1+0.5*d_v)
    axes_names = ("fractional horizontal coordinate", 
                  "fractional vertical coordinate")

    if num_axes == 4:
        Y_dim, X_dim = numpy_array.shape[:2]
        axes_sizes = (X_dim, Y_dim) + axes_sizes
        axes_scales = (1, 1) + axes_scales
        axes_offsets = (0, 0) + axes_offsets
        axes_names = ("$X$", "$Y$") + axes_names

    for axis_idx in range(num_axes):
        signal.axes_manager[axis_idx].size = axes_sizes[axis_idx]
        signal.axes_manager[axis_idx].scale = axes_scales[axis_idx]
        signal.axes_manager[axis_idx].offset = axes_offsets[axis_idx]
        signal.axes_manager[axis_idx].name = axes_names[axis_idx]

    return signal



attr_name = "convergence_map_of_distorted_then_resampled_images"
convergence_map = getattr(distortion_model, attr_name)

kwargs = {"torch_tensor": convergence_map, 
          "title": "Convergence Map Of Distorted Then Resampled Images"}
convergence_map_signal = convert_torch_tensor_to_signal(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
convergence_map_signal.plot(**kwargs)

From this convergence map, we can conclude that either the calculation of the
flow field of the right-inverse of the coordinate transformation is not
accurate, or that the right-inverse is not well-defined, at the sampling points
in the blue regions. For the flow field of the right-inverse of the coordinate
transformation, or for images that have been distorted according to the
distortion model, users may want to mask the pixels corresponding to the blue
regions prior to performing any subsequent operations on said flow fields or
distorted images. Alternatively, if users prefer to use a rectangular mask,
distortion models also store as an attribute the specifications of the minimum
frame to mask all boolean values of ``False`` in the aforementioned convergence
map. The specifications of this mask frame are stored in the attribute
``mask_frame_of_distorted_then_resampled_images``. Let's visualize the mask
frame.

In [ ]:
attr_name = "mask_frame_of_distorted_then_resampled_images"
mask_frame = getattr(distortion_model, attr_name)

L = mask_frame[0]  # Width in units of pixels of left side of mask frame.
R = mask_frame[1]  # Width in units of pixels of right side of mask frame.
B = mask_frame[2]  # Width in units of pixels of bottom side of mask frame.
T = mask_frame[3]  # Width in units of pixels of top side of mask frame.

v_dim, h_dim = convergence_map.shape

mask = torch.ones_like(convergence_map)
mask[:T, :] = 0
mask[v_dim-B:, :] = 0
mask[:, :L] = 0
mask[:, h_dim-R:] = 0

kwargs = {"torch_tensor": mask, 
          "title": "Mask Frame Of Distorted Then Resampled Images"}
mask_signal = convert_torch_tensor_to_signal(**kwargs)



kwargs = {"torch_tensor": 1.0*mask+1.0*convergence_map, 
          "title": "Mask Frame + Convergence Map"}
mask_plus_convergence_map_signal = convert_torch_tensor_to_signal(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
mask_signal.plot(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
mask_plus_convergence_map_signal.plot(**kwargs)

### Visualizing the flow fields ###

Let's now visualize the flow fields of both the coordinate transformation, and
its right-inverse, however let's visualize only the flow field vectors at every
8th row and column of the sampling grid. To visualize the flow fields, we make
use of the attributes ``sampling_grid``, ``flow_field_of_coord_transform``, and
``flow_field_of_coord_transform_right_inverse``.

In [ ]:
slice_step = 8



quiver_kwargs = {"angles": "uv",
                 "pivot": "middle",
                 "scale_units": "width"}



attr_name = "sampling_grid"
sampling_grid = getattr(distortion_model, attr_name)
sampling_grid = (sampling_grid[0].numpy(), sampling_grid[1].numpy())

X = sampling_grid[0][::slice_step, ::slice_step]
Y = sampling_grid[1][::slice_step, ::slice_step]



fig, ax = plt.subplots()

attr_name = "flow_field_of_coord_transform"
flow_field = getattr(distortion_model, attr_name)
flow_field = (flow_field[0].numpy(), flow_field[1].numpy())

U = flow_field[0][::slice_step, ::slice_step]
V = flow_field[1][::slice_step, ::slice_step]

kwargs = quiver_kwargs
ax.quiver(X, Y, U, V, **kwargs)
ax.set_title("Flow Field Of Coordinate Transformation")
ax.set_xlabel("fractional horizontal coordinate")
ax.set_ylabel("fractional vertical coordinate")

plt.gca().set_aspect('equal')

plt.show()



fig, ax = plt.subplots()

attr_name = "flow_field_of_coord_transform_right_inverse"
flow_field = getattr(distortion_model, attr_name)
flow_field = (flow_field[0].numpy(), flow_field[1].numpy())

U = flow_field[0][::slice_step, ::slice_step]
V = flow_field[1][::slice_step, ::slice_step]

kwargs = quiver_kwargs
ax.quiver(X, Y, U, V, **kwargs)
ax.set_title("Flow Field Of Right-Inverse Of Coordinate Transformation")
ax.set_xlabel("fractional horizontal coordinate")
ax.set_ylabel("fractional vertical coordinate")

plt.gca().set_aspect('equal')

plt.show()

### Distorting then resampling images ###

Now let's distort then resample our original undistorted image set using the
method ``distort_then_resample_images``. See
[here](https://mrfitzpa.github.io/distoptica/_autosummary/distoptica.DistortionModel.html#distoptica.DistortionModel.distort_then_resample_images)
for a full discussion on the expected input and the resulting output.

In [ ]:
method_alias = distortion_model.distort_then_resample_images
kwargs = {"undistorted_images": undistorted_image_set_signal.data}
distorted_then_resampled_image_set = method_alias(**kwargs)

Let's visualize the distorted then resampled image set that we just created.

In [ ]:
func_alias = convert_torch_tensor_to_signal
kwargs = {"torch_tensor": distorted_then_resampled_image_set, 
          "title": "Distorted Then Resampled Image Set"}
distorted_then_resampled_image_set_signal = func_alias(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
distorted_then_resampled_image_set_signal.plot(**kwargs)

One can also visualize the out-of-bounds map of distorted then resampled images.

In [ ]:
out_of_bounds_map = \
    distortion_model.out_of_bounds_map_of_distorted_then_resampled_images

func_alias = convert_torch_tensor_to_signal
kwargs = {"torch_tensor": out_of_bounds_map, 
          "title": "Out-Of-Bounds Map Of Distorted Then Resampled Images"}
out_of_bounds_map_signal = func_alias(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
out_of_bounds_map_signal.plot(**kwargs)

### Undistorting then resampling images ###

Let's undistort the distorted image set that we created above using the method
``undistort_then_resample_images``. See
[here](https://mrfitzpa.github.io/distoptica/_autosummary/distoptica.DistortionModel.html#distoptica.DistortionModel.undistort_then_resample_images)
for a full discussion on the expected input and the resulting output.

In [ ]:
method_alias = distortion_model.undistort_then_resample_images
kwargs = {"distorted_images": distorted_then_resampled_image_set_signal.data}
distortion_corrected_image_set = method_alias(**kwargs)

Let's visualize the distortion-corrected image set and juxtapose it to the
original undistorted image set.

In [ ]:
func_alias = convert_torch_tensor_to_signal
kwargs = {"torch_tensor": distortion_corrected_image_set, 
          "title": "Distortion-Corrected Image Set"}
distortion_corrected_image_set_signal = func_alias(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
distortion_corrected_image_set_signal.plot(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
undistorted_image_set_signal.plot(**kwargs)

Note that the pixel values are different between the signals because the first
signal is a result of resampling, i.e. it contains more pixels. For each image
in the first signal, the sum of all pixels is approximately equal to that in the
corresponding image in the second signal, as expected.

In [ ]:
print("Pixel sums in first signal:")
print(distortion_corrected_image_set_signal.data.sum(axis=(2, 3)))

print()

print("Pixel sums in second signal:")
print(undistorted_image_set_signal.data.sum(axis=(2, 3)))

Lastly, let's visualize the out-of-bounds map of undistorted then resampled
images.

In [ ]:
out_of_bounds_map = \
    distortion_model.out_of_bounds_map_of_undistorted_then_resampled_images

func_alias = convert_torch_tensor_to_signal
kwargs = {"torch_tensor": out_of_bounds_map, 
          "title": "Out-Of-Bounds Map Of Undistorted Then Resampled Images"}
out_of_bounds_map_signal = func_alias(**kwargs)



kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
out_of_bounds_map_signal.plot(**kwargs)

### Applying the coordinate transformation directly ###

Using the coordinate transformation parameters in conjunction with the function
``distoptica.apply_coord_transform``, we can apply directly the coordinate
transformation to a set of coordinates of points in an undistorted image:

In [ ]:
# ``u_x`` and ``u_y`` together store the fractional coordinates of points in an
# undistorted image.
u_x = torch.tensor([[0.35, 0.50, 0.65],
                    [0.35, 0.50, 0.65]])
u_y = torch.tensor([[0.35, 0.35, 0.35],
                    [0.35, 0.35, 0.35]])

kwargs = {"u_x": u_x,
          "u_y": u_y,
          "coord_transform_params": coord_transform_params}
q_x, q_y = distoptica.apply_coord_transform(**kwargs)

print("q_x:")
print(q_x)

print()

print("q_y:")
print(q_y)

Similarly, we can apply the right inverse of the coordinate transformation:

In [ ]:
kwargs = \
    {"q_x": q_x,
     "q_y": q_y,
     "coord_transform_params": coord_transform_params, 
     "least_squares_alg_params": least_squares_alg_params}
u_x, u_y, convergence_map = \
    distoptica.apply_coord_transform_right_inverse(**kwargs)

print("u_x:")
print(u_x)

print()

print("u_y:")
print(u_y)

print()

print("convergence_map:")
print(convergence_map)

The tensor ``convergence_map`` stores the convergence map of the iterative
algorithm used to apply the right inverse of the coordinate transformation. As
we can see, for this example, the iterative algorithm converges for all
coordinates.